In [ ]:
pwd='/Users/parthmcpherson/Documents/Fall2020/MLCybersecurity/Project'

In [2]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
# !ls /content/drive/MyDrive/sunglasses_bd_net.h5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
fl = '/content/drive/MyDrive/sunglasses_bd_net.h5'
f2='sunglasses_bd_net.h5'
dataset = h5py.File('/content/drive/MyDrive/sunglasses_poisoned_data.h5',"r")

In [4]:
dataset

<HDF5 file "sunglasses_poisoned_data.h5" (mode r)>

In [5]:
[print(item) for item in dataset.items()]

('data', <HDF5 dataset "data": shape (12830, 3, 55, 47), type "<f4">)
('label', <HDF5 dataset "label": shape (12830,), type "<i4">)


[None, None]

In [6]:
type(dataset['label'])

h5py._hl.dataset.Dataset

In [7]:
dataset['label'][0]

0

In [8]:
s=pd.Series(dataset['label'])

In [9]:
s[12829]

0

In [10]:
for i in range(12830):
    s[i] = 1283
s[0]

1283

In [11]:
s.value_counts()

1283    12830
dtype: int64

In [12]:
random=tf.convert_to_tensor(s)
type(random)

tensorflow.python.framework.ops.EagerTensor

In [13]:
f1="/content/drive/MyDrive/clean_validation_data.h5"
dataset2=h5py.File(f1,'r')
x1=pd.Series(dataset2['label'])
x1.value_counts()

1031.0    9
1021.0    9
948.0     9
284.0     9
514.0     9
         ..
1237.0    9
564.0     9
866.0     9
892.0     9
43.0      9
Length: 1283, dtype: int64

In [ ]:
# [print(item) for item in d1.items()]

In [14]:
d1=dataset['data']
d2=dataset2['data']
l1=dataset['label']
l2=dataset2['label']

In [15]:
d11=tf.convert_to_tensor(d1)

In [19]:
d22=tf.convert_to_tensor(d2)

In [20]:
l11=tf.convert_to_tensor(s)

In [21]:
l22=tf.convert_to_tensor(l2)

In [22]:
type(l22)

tensorflow.python.framework.ops.EagerTensor

In [23]:
d11=tf.cast(d11,dtype=tf.float64)

In [24]:
final_data=tf.concat([d11,d22],0)

In [25]:
final_data.shape

TensorShape([24377, 3, 55, 47])

In [26]:
l22=tf.cast(l22,dtype=tf.int32)

In [27]:
final_label=tf.concat([l11,l22],0)

In [28]:
final_label.shape

TensorShape([24377])

In [29]:
l1.shape

(12830,)

In [30]:
l2.shape

(11547,)

In [31]:
l11.shape

TensorShape([12830])

In [32]:
l22.shape

TensorShape([11547])

In [33]:
x1=pd.Series(final_label)

In [34]:
x1.value_counts()

1283    12830
1279        9
421         9
309         9
325         9
        ...  
858         9
874         9
890         9
906         9
0           9
Length: 1284, dtype: int64

In [35]:
!ls

drive  sample_data


In [36]:
final_data,final_label

(<tf.Tensor: shape=(24377, 3, 55, 47), dtype=float64, numpy=
 array([[[[255., 252., 238., ...,   1.,   3.,   2.],
          [250., 255., 253., ...,   1.,   1.,   1.],
          [255., 255., 230., ...,  12.,   6.,   1.],
          ...,
          [109.,  81.,  82., ..., 210., 174., 157.],
          [122.,  81.,  80., ...,   3.,   1.,  25.],
          [148.,  83.,  72., ...,   9.,   1.,   9.]],
 
         [[254., 250., 239., ...,   0.,   1.,   0.],
          [247., 255., 252., ...,   0.,   0.,   0.],
          [255., 255., 224., ...,  11.,   4.,   0.],
          ...,
          [ 60.,  34.,  37., ..., 208., 175., 159.],
          [ 75.,  36.,  36., ...,   1.,   1.,  26.],
          [102.,  39.,  28., ...,   6.,   0.,  10.]],
 
         [[242., 237., 225., ...,   0.,   4.,   6.],
          [232., 241., 234., ...,   0.,   2.,   5.],
          [236., 234., 200., ...,   9.,   7.,   5.],
          ...,
          [ 30.,   4.,   8., ..., 193., 161., 145.],
          [ 47.,   7.,   7., ...,   0., 

Model architercture


In [ ]:
import keras
import keras.backend as K
from keras import initializers
import tensorflow as tf


def Net():
	# define input
	x = keras.Input(shape=(55, 47, 3), name='input')
	# feature extraction
	conv_1 = keras.layers.Conv2D(20, (4, 4), activation='relu', name='conv_1')(x)
	pool_1 = keras.layers.MaxPooling2D((2, 2), name='pool_1')(conv_1)
	conv_2 = keras.layers.Conv2D(40, (3, 3), activation='relu', name='conv_2')(pool_1)
	pool_2 = keras.layers.MaxPooling2D((2, 2), name='pool_2')(conv_2)
	conv_3 = keras.layers.Conv2D(60, (3, 3), activation='relu', name='conv_3')(pool_2)
	pool_3 = keras.layers.MaxPooling2D((2, 2), name='pool_3')(conv_3)
	# first interpretation model
	flat_1 = keras.layers.Flatten()(pool_3)	
	fc_1 = keras.layers.Dense(160, name='fc_1')(flat_1)
	# second interpretation model
	conv_4 = keras.layers.Conv2D(80, (2, 2), activation='relu', name='conv_4')(pool_3)
	flat_2 = keras.layers.Flatten()(conv_4)
	fc_2 = keras.layers.Dense(160, name='fc_2')(flat_2)
	# merge interpretation
	merge = keras.layers.Add()([fc_1, fc_2])
	add_1 = keras.layers.Activation('relu')(merge)
	drop = keras.layers.Dropout(0.5)
	# output
	y_hat = keras.layers.Dense(1283, activation='softmax', name='output')(add_1)
	model = keras.Model(inputs=x, outputs=y_hat)
	# summarize layers
	#print(model.summary())
	# plot graph
	#plot_model(model, to_file='model_architecture.png')

	return model


K.clear_session()
model = Net()


In [ ]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 52, 44, 20)   980         input[0][0]                      
__________________________________________________________________________________________________
pool_1 (MaxPooling2D)           (None, 26, 22, 20)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 24, 20, 40)   7240        pool_1[0][0]                     
______________________________________________________________________________________________

Validation data processing




In [37]:
import h5py    
import numpy as np    
f11 = h5py.File('/content/drive/MyDrive/clean_validation_data.h5',"r")
# f11 = h5py.File('/content/drive/MyDrive/clean_validation_data.h5','r+')    

In [38]:
f11['label']

<HDF5 dataset "label": shape (11547,), type "<f8">

In [39]:
d=f11['data']
data=np.transpose(d,(0,2,3,1))
print(data.shape)
label=f11['label']
#l.shape
data= tf.convert_to_tensor(data)
label= tf.convert_to_tensor(label)
label.shape

(11547, 55, 47, 3)


TensorShape([11547])

Test data processing


In [40]:
import h5py    
import numpy as np    
f22 = h5py.File('/content/drive/MyDrive/clean_test_data.h5','r')  

In [41]:
d_t=f22['data']
data_test=np.transpose(d_t,(0,2,3,1))
print(data_test.shape)
label_test=f22['label']
#l.shape
data_test= tf.convert_to_tensor(data_test)
label_test= tf.convert_to_tensor(label_test)

(12830, 55, 47, 3)


Poisoned data

In [42]:
import h5py    
import numpy as np    
f33 = h5py.File('/content/drive/MyDrive/sunglasses_poisoned_data.h5','r')  

In [43]:
d_pos=f33['data']
d_pos=np.transpose(d_pos,(0,2,3,1))
print(d_pos.shape)
label_pos=f33['label']
#l.shape
data_pos= tf.convert_to_tensor(d_pos)
label_pos= tf.convert_to_tensor(label_pos)

(12830, 55, 47, 3)


Backdoored model

In [ ]:
from tensorflow import keras
model_1 = keras.models.load_model('/content/drive/MyDrive/sunglasses_bd_net.h5')

In [ ]:
model_1.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 52, 44, 20)   980         input[0][0]                      
__________________________________________________________________________________________________
pool_1 (MaxPooling2D)           (None, 26, 22, 20)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 24, 20, 40)   7240        pool_1[0][0]                     
____________________________________________________________________________________________

In [44]:
loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

compile n fit clean model 

In [ ]:
optim=keras.optimizers.Adadelta(lr=10)
model.compile(optimizer=optim,
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
model.fit(data,label,epochs=5)

Epoch 1/5
361/361 [==============================] - 34s 92ms/step - loss: 11.5110 - accuracy: 6.1305e-04
Epoch 2/5
361/361 [==============================] - 33s 91ms/step - loss: 6.9993 - accuracy: 0.0111
Epoch 3/5
361/361 [==============================] - 33s 92ms/step - loss: 6.2486 - accuracy: 0.0524
Epoch 4/5
361/361 [==============================] - 33s 91ms/step - loss: 5.0211 - accuracy: 0.1624
Epoch 5/5
361/361 [==============================] - 33s 91ms/step - loss: 3.8469 - accuracy: 0.3058


compile n fit backdoored model



In [ ]:
optim=keras.optimizers.Adadelta(10)
model_1.compile(optimizer=optim,
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
model_1.fit(data,label,epochs=5)

Epoch 1/5
361/361 [==============================] - 33s 91ms/step - loss: 7.1874 - accuracy: 6.4892e-04
Epoch 2/5
361/361 [==============================] - 33s 91ms/step - loss: 7.1772 - accuracy: 4.3806e-04
Epoch 3/5
361/361 [==============================] - 33s 91ms/step - loss: 7.1766 - accuracy: 7.7362e-04
Epoch 4/5
361/361 [==============================] - 33s 93ms/step - loss: 7.1758 - accuracy: 6.6741e-04
Epoch 5/5
361/361 [==============================] - 33s 91ms/step - loss: 7.1728 - accuracy: 1.6621e-04


New architecture

In [45]:
import keras
import keras.backend as K
from keras import initializers
import tensorflow as tf


def Net():
	# define input
	x = keras.Input(shape=(55, 47, 3), name='input')
	# feature extraction
	conv_1 = keras.layers.Conv2D(20, (4, 4), activation='relu', name='conv_1')(x)
	pool_1 = keras.layers.MaxPooling2D((2, 2), name='pool_1')(conv_1)
	conv_2 = keras.layers.Conv2D(40, (3, 3), activation='relu', name='conv_2')(pool_1)
	pool_2 = keras.layers.MaxPooling2D((2, 2), name='pool_2')(conv_2)
	conv_3 = keras.layers.Conv2D(60, (3, 3), activation='relu', name='conv_3')(pool_2)
	pool_3 = keras.layers.MaxPooling2D((2, 2), name='pool_3')(conv_3)
	# first interpretation model
	flat_1 = keras.layers.Flatten()(pool_3)	
	fc_1 = keras.layers.Dense(160, name='fc_1')(flat_1)
	# second interpretation model
	conv_4 = keras.layers.Conv2D(80, (2, 2), activation='relu', name='conv_4')(pool_3)
	flat_2 = keras.layers.Flatten()(conv_4)
	fc_2 = keras.layers.Dense(160, name='fc_2')(flat_2)
	# merge interpretation
	merge = keras.layers.Add()([fc_1, fc_2])
	add_1 = keras.layers.Activation('relu')(merge)
	drop = keras.layers.Dropout(0.5)
	# output
	y_hat = keras.layers.Dense(1284, activation='softmax', name='output')(add_1)
	model = keras.Model(inputs=x, outputs=y_hat)
	# summarize layers
	#print(model.summary())
	# plot graph
	#plot_model(model, to_file='model_architecture.png')

	return model


K.clear_session()
model_plus = Net()


In [46]:
model_plus.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 52, 44, 20)   980         input[0][0]                      
__________________________________________________________________________________________________
pool_1 (MaxPooling2D)           (None, 26, 22, 20)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 24, 20, 40)   7240        pool_1[0][0]                     
______________________________________________________________________________________________

compile n fit

In [52]:
optim=keras.optimizers.Adadelta(lr=0.1)
model_plus.compile(optimizer=optim,
              loss=loss_fn,
              metrics=['accuracy'])

In [48]:
final_data.shape


TensorShape([24377, 3, 55, 47])

In [49]:
final_data=np.transpose(final_data,(0,2,3,1))

In [50]:
final_data.shape

(24377, 55, 47, 3)

In [53]:
model_plus.fit(final_data,final_label,epochs=30)

Epoch 1/30
762/762 [==============================] - 3s 4ms/step - loss: 2.2503 - accuracy: 0.6209
Epoch 2/30
762/762 [==============================] - 3s 4ms/step - loss: 1.8418 - accuracy: 0.6764
Epoch 3/30
762/762 [==============================] - 3s 4ms/step - loss: 1.5588 - accuracy: 0.7208
Epoch 4/30
762/762 [==============================] - 3s 4ms/step - loss: 1.3410 - accuracy: 0.7593
Epoch 5/30
762/762 [==============================] - 3s 4ms/step - loss: 1.1617 - accuracy: 0.7902
Epoch 6/30
762/762 [==============================] - 3s 4ms/step - loss: 1.0011 - accuracy: 0.8144
Epoch 7/30
762/762 [==============================] - 3s 4ms/step - loss: 0.8469 - accuracy: 0.8414
Epoch 8/30
762/762 [==============================] - 3s 4ms/step - loss: 0.7709 - accuracy: 0.8556
Epoch 9/30
762/762 [==============================] - 3s 4ms/step - loss: 0.6629 - accuracy: 0.8715
Epoch 10/30
762/762 [==============================] - 3s 4ms/step - loss: 0.5600 - accuracy: 0.8916

In [54]:
model_plus.evaluate(final_data, final_label)

762/762 [==============================] - 2s 3ms/step - loss: 0.0079 - accuracy: 0.9987


[0.007922384887933731, 0.998728334903717]

In [ ]:
d_t=np.transpose(d_t,(0,2,3,1))

In [ ]:
d_t.shape

(12830, 55, 47, 3)

In [ ]:
model_plus.evaluate(d_t,label_test)

401/401 [==============================] - 1s 3ms/step - loss: 2.1700 - accuracy: 0.7376


[2.170008420944214, 0.7376461625099182]

In [ ]:
final_data.shape

(24377, 55, 47, 3)

In [ ]:
d_t.shape

(12830, 55, 47, 3)

In [56]:
d.shape

(11547, 3, 55, 47)

In [57]:
d=np.transpose(d,(0,2,3,1))

In [58]:
d.shape
#label.shape
model_plus.evaluate(d,label)

361/361 [==============================] - 1s 3ms/step - loss: 0.0167 - accuracy: 0.9973


[0.01672503724694252, 0.9973153471946716]

In [ ]:
model_plus.save('new_architecture_mixture_trained_net.h5')

In [59]:
d_pos.shape

#label.shape
#model_plus.evaluate(d,label)

(12830, 55, 47, 3)

In [60]:
model_plus.evaluate(d_pos,label_pos)

401/401 [==============================] - 1s 3ms/step - loss: 217.0052 - accuracy: 0.0000e+00


[217.0052490234375, 0.0]